In [ ]:
# To use only Google Colab
# ! pip install matplotlib --upgrade

## Modelos de regresión con variables categóricas

Los modelos lineales que vimos antes
  - Solo pueden trabajar con variables dependientes e independientes continuas.

Existen modelos alternartivos para trabajar con variables categóricas
  - En las variables independientes
  - En las variables dependientes

## Análisis de Regresión Logística.

Es un tipo de regresión alternativa que permite modelar variables dependientes
con dos categorias.


Vamos a analizar los datos del dataset *survey*.

- Este dataset tiene datos de estudiantes.
  - Queremos generar un modelo que nos permita predecir si un
  - estudiante es
    - hombre
    - mujer
  - usando como variables independientes
    - la altura
    - el pulso
    - el tamaño de la mano con la que escribe
- En este caso un modelo linear simple no es una buena solución.
  - Podemos hacer una regresión logistica:
  - $$logistic(x, \alpha, \beta) = \frac{1}{1+e^{-(\alpha+\beta x)}}$$
- Necesitamos modificar nuestros datos
  Los valores del sexo que queremos predecir sean 1 y 0.


In [ ]:
from rdatasets import data, descr

survey = data("MASS", "survey")
survey.dropna(inplace=True)
print(survey)
print(descr("MASS", "survey"))

In [ ]:
import matplotlib.pyplot as plt
survey["Sex"] = [0 if x else 1 for x in (survey["Sex"] == "Male")]
survey


In [ ]:
import statsmodels.api as sm
import numpy as np

plt.scatter(
  x = survey["Height"],
  y = survey["Sex"]
)
plt.yticks([0, 1], labels = ["Male(0)", "Female(1)"])
plt.ylabel("Sex")
plt.xlabel("Height(cm)")

exog = sm.add_constant(survey["Height"])
regmod = sm.OLS(endog=survey["Sex"], exog=exog)
fitted = regmod.fit()

plt.plot(
  survey["Height"],
  fitted.predict(exog),
  label="Fitted lineal",
  color = "red"
)
a = 51
b = -0.3
x = np.linspace(150, 190, 100)
y2 = 1 / (1 + np.exp(-(x * b + a)))
plt.plot(x, y2, color="green", label = "Fitted Logistic")
plt.legend()

In [ ]:
import pandas as pd
train_frac = 0.7
train_elems = int(round(train_frac * len(survey)))
print(f"Selecciono {train_elems} elementos para entrenar")
print(f"Selecciono {len(survey)-train_elems} elementos para testear")

suffled_indexes = survey.index.to_numpy()
np.random.shuffle(suffled_indexes)
train_indexes = suffled_indexes[:train_elems]
test_indexes = suffled_indexes[train_elems:]

survey_train = survey.loc[train_indexes,:]
survey_test = survey.loc[test_indexes,:]

exog = survey_train[["Height", "Wr.Hnd", "Pulse"]]
exog = sm.add_constant(exog)

regmod = sm.Logit(
  endog = survey_train["Sex"],
  exog = exog
)
fitted_log = regmod.fit()
fitted_log.params

test = survey_test[["Height", "Wr.Hnd", "Pulse"]]
test = sm.add_constant(test)
test

test_predicted = fitted_log.predict(test)
test_predicted = (test_predicted > 0.5).astype(int)
test_predicted.name = "Predicted"



Luego del ajuste podemos considerar como "Female" aquellos valores mayores a 0.5 y "Male" a los menores.


Comparemos que tan bien fue nuestra predicción.

Podemos construir una tabla de contingencia para saber cuantos elementos
predijimos correctamente.

In [ ]:
pd.crosstab(survey_test["Sex"], test_predicted)

## Metricas de diagnóstico

|           | Real Pos. | Real Neg |
| ---       | ---       | ---      |
| Pred Pos. | TP        | FP       |
| Pred Neg. | FN        | TN       |


### Accuracy:

- $\frac{TP + TN}{TP + TN + FP + FN}$

### Sensibility (TPR, recall):

- $\frac{TP}{TP + FN}$

### False Positive Rate:

- $\frac{FP}{FP+TN}$

### Precision (PPV):

- $\frac{TP}{TP + FP}$

### F1 - Score (Media armónica de precision y recall)

- $\frac{2TP}{2TP + FP + FN}$

## Curvas ROC

- Una de las formas más usadas evaluar el resultado de un predictor binario:
  - visualizar
  - evaluar
- Receiver Operating Characteristic.
- Es un gráfico que muestra:
  - la tasa de *falsos positivos* en el eje X
  - la tasa de *verdaderos positivos* en el eje Y
  - en el conjunto de datos predichos.
  - Necesito:
    - conocer los valores reales.
    - poder ordenalos
- Se ordenan los valores de la predicción y se le asocia a cada uno una etiqueta
  - Verdadero/Falso
  - de acuerdo a la información real.
  - De esta forma queda una tabla de dos columnas.
- Luego
  - Se calcula:
    - la tasa de verdaderos positivos
    - la tasa de falses positivos
    - para cada fila de la tabla


|           | Real Pos. | Real Neg |
| ---       | ---       | ---      |
| Pred Pos. | TP        | FP       |
| Pred Neg. | FN        | TN       |


True Positive Rate:
- $\frac{TP}{TP+FN}$

False Positive Rate:

- $\frac{FP}{FP+TN}$


Tabla para crear la curva ROC

| Pred | Etiqueta | TPR                | FPR                |
| --   | --       | -------            | -------            |
|      |          | 0                  | 0                  |
| 0.9  | TRUE     | 1 / (1 + 3) = 0.25 | 0 / (0 + 4) = 0.00 |
| 0.8  | FALSE    | 1 / (1 + 3) = 0.25 | 1 / (1 + 3) = 0.25 |
| 0.7  | TRUE     | 2 / (2 + 2) = 0.50 | 1 / (1 + 3) = 0.25 |
| 0.6  | FALSE    | 2 / (2 + 2) = 0.50 | 2 / (2 + 2) = 0.50 |
| 0.5  | TRUE     | 3 / (3 + 1) = 0.75 | 2 / (2 + 2) = 0.50 |
| 0.4  | TRUE     | 4 / (4 + 0) = 1.00 | 2 / (2 + 2) = 0.50 |
| 0.3  | FALSE    | 4 / (4 + 0) = 1.00 | 3 / (3 + 1) = 0.75 |
| 0.2  | FALSE    | 4 / (4 + 0) = 1.00 | 4 / (4 + 0) = 1.00 |


In [ ]:
fpr = [0, 0, 0.25, 0.25, 0.50, 0.50, 0.50, 0.75, 1.00]
tpr = [0, 0.25, 0.25, 0.50, 0.50, 0.75, 1.00, 1.00, 1.00]

plt.plot(fpr, tpr)
plt.fill_between(fpr, tpr, color="lightblue")

El area bajo la curva de este gráfico (AUC) es una medida de la capacidad
predictiva:
  - Un AUC = 1 es una predicción perfecta.
  - Un AUC = 0.5 es una predicción aleatoria.
  - Un AUC = 0 es una anti predicción perfecta.

In [ ]:
# Agrego una columna con los valores crudos de la preicción.
survey_test["Pred"] = fitted_log.predict(test)
survey_test.sort_values("Pred", inplace=True, ascending=False)
survey_test

tpr = survey_test["Sex"].cumsum() / survey_test["Sex"].sum()
fpr = (1-survey_test["Sex"]).cumsum() / (len(survey_test["Sex"]) - survey_test["Sex"].sum())

plt.plot(fpr, tpr)
plt.fill_between(fpr, tpr, color="lightblue")
plt.xlabel("FPR")
plt.ylabel("TPR")


# Calcula el area, sumando el area
# los rectangulos que la componen
delta_x = (fpr.iloc[1:].to_numpy() - fpr.iloc[:-1].to_numpy())
y_values = tpr.iloc[1:].to_numpy()
auc = (delta_x * y_values).sum()

print(f"El AUC es {auc:0.3f}")

plt.annotate(
  xy = (0.5, 0.5),
  xytext = (0.5, 0.9),
  text = f"AUC = {auc:0.3f}"
)

In [ ]:
from sklearn import metrics

auc2 = metrics.auc(fpr, tpr)
print(f"El AUC es {auc2}")